In [1]:
import os
import sys
import yaml
import types
import pprint
import logging
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.memory.canvas import TextCanvasMemory
from autogen_agentchat import EVENT_LOGGER_NAME, TRACE_LOGGER_NAME
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_core import SingleThreadedAgentRuntime
from autogen_ext.models.openai import OpenAIChatCompletionClient
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor


# Project import paths
# Get the current directory (which is project/test)
notebook_dir = os.getcwd()

# Compute absolute paths for lib and src (one level up from test/)
lib_path = os.path.abspath(os.path.join(notebook_dir, '..', 'lib'))
src_path = os.path.abspath(os.path.join(notebook_dir, '..', 'src'))

sys.path.append(lib_path)
sys.path.append(src_path)

from agents_creator import create_agents_from_config
from rag_helpers import rag_indexer_runner
from model_loader import load_model_client


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
logging.basicConfig(level=logging.WARNING)

# For trace logging.
trace_logger = logging.getLogger(TRACE_LOGGER_NAME)
trace_logger.addHandler(logging.StreamHandler())
trace_logger.setLevel(logging.DEBUG)

# For structured message logging, such as low-level messages between agents.
event_logger = logging.getLogger(EVENT_LOGGER_NAME)
event_logger.addHandler(logging.StreamHandler())
event_logger.setLevel(logging.DEBUG)

In [3]:
# Set up telemetry span exporter.
otel_exporter = OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)
span_processor = BatchSpanProcessor(otel_exporter)

# Set up telemetry trace provider.
tracer_provider = TracerProvider(resource=Resource({"service.name": "autogen-test-agentchat"}))
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)

# Instrument the OpenAI Python library
OpenAIInstrumentor().instrument()

# we will get reference this tracer later using its service name
# tracer = trace.get_tracer("autogen-test-agentchat")

In [4]:
rag_vector_memory = await rag_indexer_runner('/run/media/jun/Jun 6T/Code/TFM_BA_GPT/lib/rag_config.yaml')

✅ All documents and URLs are already indexed.


In [5]:
# 1. Create the canvas memory and tool:
text_canvas_memory = TextCanvasMemory()
update_file_tool = text_canvas_memory.get_update_file_tool()

# 2. Register the tool as 'canvas.update_file_tool' for YAML import:
canvas = types.SimpleNamespace()
canvas.update_file_tool = update_file_tool
sys.modules['canvas'] = canvas  # This enables the tool import by your agent factory!

In [6]:
model_client = load_model_client("azure")

In [7]:
# Construct the absolute path to project/lib/agents_config.yaml
AGENTS_CONFIG = os.path.abspath(os.path.join(notebook_dir, '..', 'lib', 'agents_config.yaml'))
with open(AGENTS_CONFIG, "r") as f:
        config_yaml = yaml.safe_load(f)
agent_names = list(config_yaml.keys())

user_proxy, agents = await create_agents_from_config(
    AGENTS_CONFIG,
    agent_names=agent_names,
    model_client=model_client,
    vector_memory=rag_vector_memory,
    canvas_memory=text_canvas_memory,
    input_func=input
)

In [8]:
for name, agent in agents.items():
    print(f"\n--- Agent: {name} ---")
    pprint.pprint(vars(agent))




--- Agent: planning_agent ---
{'_description': 'The orchestrator agent responsible for breaking down a new '
                 'task into subtasks, assigning them to the right agents, and '
                 'ensuring the workflow progresses smoothly until completion.',
 '_handoff_tools': [],
 '_handoffs': {},
 '_is_running': False,
 '_max_tool_iterations': 1,
 '_memory': None,
 '_metadata': {},
 '_model_client': <autogen_ext.models.openai._openai_client.AzureOpenAIChatCompletionClient object at 0x7f545592c1a0>,
 '_model_client_stream': True,
 '_model_context': <autogen_core.model_context._unbounded_chat_completion_context.UnboundedChatCompletionContext object at 0x7f545592cd70>,
 '_name': 'planning_agent',
 '_output_content_type': None,
 '_output_content_type_format': None,
 '_reflect_on_tool_use': False,
 '_structured_message_factory': None,
 '_system_messages': [SystemMessage(content='You are the PLANNING AGENT, the coordinator for this multi-agent system.\n\n**YOUR MISSION**\n  Brea

In [9]:
for name, agent in agents.items():
    print(f"\n--- Agent: {name} ---")
    print("Tools:")
    for tool in agent._tools:
        print(f"Tool Name: {tool.name}")
        print(f"Description: {tool.description}\n")



--- Agent: planning_agent ---
Tools:

--- Agent: interviewer_agent ---
Tools:
Tool Name: save_conversation_to_jsonl
Description: 
Saves a single conversation turn (speaker + message) to a jsonl file.

Args:
    speaker (str): Who is speaking (e.g., "interviewer" or "user").
    message (str): The content of the message.



--- Agent: summarizer_agent ---
Tools:
Tool Name: save_summary_to_markdown
Description: 
Saves the final confirmed summary to a Markdown file.

Args:
    content (str): The final summary text, formatted in Markdown.



--- Agent: functional_spec_writer_agent ---
Tools:
Tool Name: write_pdf_file_tool
Description: 
Writes the given text into a PDF file.

Args:
    text (str): The full text content to write into the PDF.
    file_path (str): Optional custom path for saving. Defaults to DEFAULT_PDF_PATH.

Returns:
    str: Confirmation or error message.



--- Agent: functional_spec_checker_agent ---
Tools:
Tool Name: read_pdf_file_tool
Description: 
Reads the entire co

In [10]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [11]:
# Order agents (workflow order), user_proxy last
agent_order = [
    "planning_agent",
    "interviewer_agent",
    "summarizer_agent",
    "functional_spec_writer_agent",
    "functional_spec_checker_agent",
    "diagram_creator_agent",
    "mermaid_code_reviewer_agent"
]
member_agents = [agents[name] for name in agent_order if name in agents] + [user_proxy]

In [12]:
termination = MaxMessageTermination(100000000) | TextMentionTermination("TERMINATE")

# --- Use SelectorGroupChat ---
team = SelectorGroupChat(
    member_agents,
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False
)

In [13]:
task = "Interview a customer to collect requirements and scope of a new product then write the functional specifications"

In [14]:
runtime = SingleThreadedAgentRuntime(
            tracer_provider=trace.NoOpTracerProvider(),  # Disable telemetry for runtime.
        )
runtime.start()

In [15]:
await Console(team.run_stream(task=task))

await runtime.stop()

await model_client.close()

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:269: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: planning_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: planning_agent (attempt 1)
Selected speaker: planning_agent
DEBUG:autogen_agentchat:Selected speaker: planning_agent


---------- TextMessage (user) ----------
Interview a customer to collect requirements and scope of a new product then write the functional specifications
---------- ModelClientStreamingChunkEvent (planning_agent) ----------
**TASK INITIALIZED**

Breaking the task down into manageable sub-tasks:

1. **Conduct an Interview**: Gather detailed functional requirements and scope by interviewing the customer.
2. **Summarize Findings**: Summarize all gathered information.
3. **Write Functional Specification**: Draft a functional specification document based on the gathered requirements.
4. **Review Functional Specification**: Ensure the specification is complete and clear.
5. **Create Diagrams**: Develop process flow diagrams to visually represent the product's functionality.
6. **Review Diagrams**: Validate the correctness and clarity of the diagrams.

**TASK ASSIGNMENT**

1. interviewer_agent: Start gathering requirements from the user.

**UPDATED STATUS**
- interviewer_agent: pending
- summ

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:269: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: interviewer_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: interviewer_agent (attempt 1)
Selected speaker: interviewer_agent
DEBUG:autogen_agentchat:Selected speaker: interviewer_agent


---------- MemoryQueryEvent (interviewer_agent) ----------
[MemoryContent(content='• \nFunctional Requirements Specification \n• \n1. Document Information  \no 1.1. Document Approvers & Reviewers \no 1.2. Document History \no 1.3. Reference Documents \no 1.4. Definitions, Acronyms and Abbreviations \n• \n2. Executive Summary  \no 2.1. Overview \no 2.2. Business Benefits \n• \n3. Scope  \no 3.1. In Scope \no 4.2. Out of Scope \no 3.3. Systems & Platforms LeanIX Meta-Data \n• \n4. Requirements  \no 4.1. Functional Requirements \no 4.2. Non-Functional Requirements \n• \n5. Functional Solution', mime_type='MemoryMimeType.TEXT', metadata={'chunk_index': 3, 'mime_type': 'MemoryMimeType.TEXT', 'source': 'Functional Requirements Specification Template.doc.pdf', 'score': 0.6848887205123901, 'id': '449a8a70-4043-4372-b176-0e7f32232870'}), MemoryContent(content='Functional Requirements Specification - \nTemplate \n \nFunctional Requirements Specification \nBS-[number] [title] \nDOCUMENTATION \nDo

id='d07edbb5-8d8e-409f-9680-af93eb517600' source='interviewer_agent' models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0) metadata={} created_at=datetime.datetime(2025, 8, 1, 22, 57, 26, 643781, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_G047CSvudElLkZBidPgZ8W4D', arguments='{"speaker":"interviewer","message":"Could you briefly describe the main purpose of the product?"}', name='save_conversation_to_jsonl')] type='ToolCallRequestEvent'
id='d07edbb5-8d8e-409f-9680-af93eb517600' source='interviewer_agent' models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0) metadata={} created_at=datetime.datetime(2025, 8, 1, 22, 57, 26, 643781, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_G047CSvudElLkZBidPgZ8W4D', arguments='{"speaker":"interviewer","message":"Could you briefly describe the main purpose of the product?"}', name='save_conversation_to_jsonl')] type='ToolCallRequestEvent'
DEBUG:autogen_agentchat.events:id='d07edbb5-8d8e-409f-9680-af9

...


Mermaid diagram written to disk_diagram.mmd---------- ToolCallRequestEvent (mermaid_code_reviewer_agent) ----------



id='936c87f0-b477-4a26-92e6-b932bdf1f04c' source='mermaid_code_reviewer_agent' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 1, 23, 18, 23, 678550, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='None', name='write_mermaid_to_file_tool', call_id='call_V4oEaOL3UjFciflt1ijqQfEQ', is_error=False)] type='ToolCallExecutionEvent'
id='936c87f0-b477-4a26-92e6-b932bdf1f04c' source='mermaid_code_reviewer_agent' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 1, 23, 18, 23, 678550, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='None', name='write_mermaid_to_file_tool', call_id='call_V4oEaOL3UjFciflt1ijqQfEQ', is_error=False)] type='ToolCallExecutionEvent'
DEBUG:autogen_agentchat.events:id='936c87f0-b477-4a26-92e6-b932bdf1f04c' source='mermaid_code_reviewer_agent' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 1, 23, 18, 23, 678550, tzinfo=datetime.timezone.utc) content=[FunctionExecutio

[FunctionCall(id='call_V4oEaOL3UjFciflt1ijqQfEQ', arguments='{"mermaid_code":"%% Functional Specification Diagram: AI Dialer for Travel Packages\\n%% Autogenerated based on functional specification document\\n\\nflowchart TD\\n    A[Start] --> B{Fetch customer contact data from CRM}\\n    B -->|Data Retrieved| C{Initiate Call to Customer}\\n    B -->|Error: Missing Data| Z[Stop Process / End Call]\\n    \\n    C -->|Customer Accepts Offer| D[Transfer Call to Human Agent]\\n    C -->|Customer Declines Offer| E[Does the AI need to retry?]\\n    C -->|Customer Did Not Respond| Z\\n\\n    E -->|Yes: Retries Left| F[AI Attempts to Convince Customer]\\n    E -->|No: No Retries Left| Z\\n\\n    F --> C[Re-initiate Offer Process]\\n    \\n    D --> G[Human Agent Finalizes Deal]\\n    G --> H[Record Deal Outcome in Database]\\n\\n    Z --> H[Record Deal Outcome in Database]\\n    H --> I[End Interaction Session]","filename":"disk_diagram.mmd"}', name='write_mermaid_to_file_tool')]
---------- To

/run/media/jun/Jun 6T/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:269: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)
Model selected a valid name: planning_agent (attempt 1)
DEBUG:autogen_agentchat:Model selected a valid name: planning_agent (attempt 1)
Selected speaker: planning_agent
DEBUG:autogen_agentchat:Selected speaker: planning_agent


---------- ModelClientStreamingChunkEvent (planning_agent) ----------
**STATUS UPDATE**
- interviewer_agent: done
- summarizer_agent: done
- functional_spec_writer_agent: done
- functional_spec_checker_agent: done
- diagram_creator_agent: done
- mermaid_code_reviewer_agent: done

**SUMMARY OF RESULTS**
1. Requirements were gathered through structured interviews with the user.
2. A well-documented functional specification was created and refined iteratively.
3. The specification was reviewed and finalized for completeness and clarity.
4. A process flow diagram was created in Mermaid format to visually represent the system's functionality and was revised based on review feedback.
5. The final diagram code was confirmed as complete and accurate.

**TASK COMPLETED SUCCESSFULLY**

**TERMINATE**
